### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using aql.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Runing aql in iJava
We can run aql commands in a file. No output will be displayed. 
In _trucate_test.aql_, we have a single AQL command: TRUNCATE test



In [2]:
%sh aql -c "truncate test" -h "3.235.121.162"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>6.0.0</version>
  </dependency>
</dependencies>

####  Add required Java Client Imports

These are some of the Aerospike Java Client imports needed to start developing our Application interactively.  We will add others, as needed, as we develop our solution.

In [4]:
//Require Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


#### Connect to the Aerospike Server
Instantiate the client object. Let us write a record and read it back.
We have a namespace **_test_** pre-defined on the server.

In [5]:
AerospikeClient client = new AerospikeClient("3.235.121.162", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);  //Will show the 20 byte digest

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1


#### Test Record Data Model

![fig_DM_1](./graphics/Fraud_DM_1.png)

In [6]:
//Insert a record with two bins, b1 with string v1 and b2 with integer 2 as data.

WritePolicy wPolicy = new WritePolicy();

Bin b1 = new Bin("b1", Value.get("val1"));
Bin b2 = new Bin("b2", Value.get(28));

client.put(wPolicy, key, b1, b2);

In [7]:
//Read the record that we just inserted

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Read back the record.
Record values are:
(gen:1),(exp:416944590),(bins:(b1:val1),(b2:28))


In [8]:
//Check 
System.out.println( client.getClusterStats() );

nodes(inUse,inPool,opened,closed):
C1 3.235.121.162 3000 sync(0,0,1,0) async(null)
A1 44.200.159.121 3000 sync(0,1,2,0) async(null)
B1 44.211.243.226 3000 sync(0,0,1,0) async(null)
threadsInUse: 0
recoverQueueSize: 0
invalidNodeCount: 0


In [15]:
import com.aerospike.client.cluster.ClusterStats;
import com.aerospike.client.cluster.NodeStats;
import com.aerospike.client.cluster.Node;
import com.aerospike.client.cluster.ConnectionStats;
import com.aerospike.client.async.EventLoopStats;

NodeStats[] nodes = {};
EventLoopStats[] eventLoops = {};
int threadsInUse = -1;
int recoverQueueSize = -1;
int invalidNodeCount = -1;

ClusterStats cst = new ClusterStats(nodes, eventLoops, threadsInUse,recoverQueueSize,invalidNodeCount);

cst = client.getClusterStats();
System.out.println( "Threads In Use:" + cst.threadsInUse);
System.out.println( "Recover queue size:" + cst.recoverQueueSize);
System.out.println( "Invalid Node Count:" + cst.invalidNodeCount);
System.out.println(cst.nodes.length);
int nNodes = cst.nodes.length;
for(int i=0;i<nNodes;i++){
  //System.out.println(cst.nodes[i]);
  System.out.println(cst.nodes[i].node.getName());
  System.out.println(cst.nodes[i].node.getAddress());  //Returns InetSocketAddress object
  System.out.println(cst.nodes[i].node.getAddress().getAddress().getHostAddress());  
  //InetSocketAddress member, getHostAddress() removes the forward slash.
  System.out.println(cst.nodes[i].node.getAddress().getPort()); //InetSocketAddress member
  System.out.println("sync: closed: "+cst.nodes[i].sync.closed);
  System.out.println("sync: inPool: "+cst.nodes[i].sync.inPool);  
  System.out.println("sync: inUse: "+cst.nodes[i].sync.inUse);
  System.out.println("sync: opened: "+cst.nodes[i].sync.opened);
}

Threads In Use:0
Recover queue size:0
Invalid Node Count:0
3
C1
/3.235.121.162:3000
3.235.121.162
3000
sync: closed: 0
sync: inPool: 0
sync: inUse: 0
sync: opened: 1
A1
/44.200.159.121:3000
44.200.159.121
3000
sync: closed: 1
sync: inPool: 0
sync: inUse: 0
sync: opened: 2
B1
/44.211.243.226:3000
44.211.243.226
3000
sync: closed: 0
sync: inPool: 0
sync: inUse: 0
sync: opened: 1
